### Plan of Attack:

1. Get list of new songs after a specific date
2. Get data needed for those songs
3. Create parquet table
4. Save to S3

In [ ]:
# import spotipy
# import os
# %run './credentials/settings.py'
# from spotipy.oauth2 import SpotifyOAuth
# scope = "user-library-read"

# def user_auth(scope=scope):
#     SPOTIPY_CLIENT_ID = os.getenv("SPOTIPY_CLIENT_ID")
#     SPOTIPY_CLIENT_SECRET = os.getenv("SPOTIPY_CLIENT_SECRET")
#     SPOTIPY_REDIRECT_URI = os.getenv('SPOTIPY_REDIRECT_URI_READ')
#     print()
#     sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
#     return sp
# sp = user_auth()

In [1]:
!pip install spotipy

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached spotipy-2.19.0-py3-none-any.whl (27 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: six
    Found existing installation: six 1.14.0
    Uninstalling six-1.14.0:
      Successfully uninstalled six-1.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
aiobotoc

In [10]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import os
import pandas as pd
import numpy as np
%run '../../credentials/settings.py'
from collections import defaultdict

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))

In [4]:
new_releases = sp.new_releases(country='US')

### list items we want from each album, will need to figure out how to get track ids from each album (even if an artist releases a single their is an "album" associated)
- release_date
- total_tracks
- id
- name
- artist(s)
- artist ids

In [5]:
def album_metadata(album_items):
    album_data = {}
    album_data['id'] = []
    album_data['release_date'] = []
    album_data['total_tracks']  = []
    album_data['artist_id_1'] = []
    album_data['artist_ids'] = []
    album_data['artist_names'] = []
    
    for album_item in album_items:
    
        # map 1:1 data for each record
        album_data['id'].append(album_item['id'])
        album_data['release_date'].append(album_item['release_date'])
        album_data['total_tracks'].append(album_item['total_tracks'])

        # get artist(s) data
        artist_lst = album_item['artists']
        artist_ids = [x['id'] for x in artist_lst]
        album_data['artist_id_1'].append(artist_ids[0])
        album_data['artist_ids'].append(str(artist_ids))
        album_data['artist_names'].append(str([x['name'] for x in artist_lst]))
    
    return pd.DataFrame.from_dict(album_data,orient='columns')

In [6]:
#test it
album_df = album_metadata(new_releases['albums']['items'])
album_df

,id,release_date,total_tracks,artist_id_1,artist_ids,artist_names
0,7jMo8dhMIorHoLlzTisjJy,2021-08-13,1,56oDRnqbIiwx4mymNEv7dS,"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo...","['Lizzo', 'Cardi B']"
1,2TvfE8CY37OQIPVGcWYpEA,2021-08-12,1,2o5jDhtHVPhrJdv3cEQ99Z,"['2o5jDhtHVPhrJdv3cEQ99Z', '790FomKkXshlbRYZFt...","['Tiësto', 'KAROL G']"
2,5D8Rdb09BkmHscEGSWAlA6,2021-08-13,1,3PhoLpVuITZKcymswpck5b,"['3PhoLpVuITZKcymswpck5b', '6M2wZ9GZgrQXHCFfjv...","['Elton John', 'Dua Lipa', 'PNAU']"
3,00fcDjvEq4elj756TESO0c,2021-08-13,1,78rUTD7y6Cy67W1RVzYs7t,['78rUTD7y6Cy67W1RVzYs7t'],['PinkPantheress']
4,1qMFjBarjO2xD15BwXZguD,2021-08-13,11,0C0XlULifJtAgn6ZNCW2eu,['0C0XlULifJtAgn6ZNCW2eu'],['The Killers']
5,6vqNFOfIgqe0l2Ui47YSyO,2021-08-13,6,2cFrymmkijnjDg9SS92EPM,['2cFrymmkijnjDg9SS92EPM'],['blackbear']
6,4ih8LPpqdYPezAMaPAHJYT,2021-08-07,1,46SHBwWsqBkxI7EeeBEQG7,"['46SHBwWsqBkxI7EeeBEQG7', '45TgXXqMDdF8BkjA83...","['Kodak Black', 'Rod Wave']"
7,3k4lUXBZxjUGFFQvy8PsHn,2021-08-12,1,6TIYQ3jFPwQSRmorSezPxX,['6TIYQ3jFPwQSRmorSezPxX'],['Machine Gun Kelly']
8,4pWo1MFBvVAfhjL4aPZFaD,2021-08-13,1,1vyhD5VmyZ7KMfW5gqLgo5,['1vyhD5VmyZ7KMfW5gqLgo5'],['J Balvin']
9,7L8IHgiomfEpxOm61vgPTm,2021-08-13,12,7z5WFjZAIYejWy0NI5lv4T,['7z5WFjZAIYejWy0NI5lv4T'],['Dan + Shay']


In [9]:
album_id = album_df.iloc[0,0]
album_id

'7jMo8dhMIorHoLlzTisjJy'

In [11]:
test = sp.album_tracks(album_id)
test

{'href': 'https://api.spotify.com/v1/albums/7jMo8dhMIorHoLlzTisjJy/tracks?offset=0&limit=50',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/56oDRnqbIiwx4mymNEv7dS'},
     'href': 'https://api.spotify.com/v1/artists/56oDRnqbIiwx4mymNEv7dS',
     'id': '56oDRnqbIiwx4mymNEv7dS',
     'name': 'Lizzo',
     'type': 'artist',
     'uri': 'spotify:artist:56oDRnqbIiwx4mymNEv7dS'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/4kYSro6naA4h99UJvo89HB'},
     'href': 'https://api.spotify.com/v1/artists/4kYSro6naA4h99UJvo89HB',
     'id': '4kYSro6naA4h99UJvo89HB',
     'name': 'Cardi B',
     'type': 'artist',
     'uri': 'spotify:artist:4kYSro6naA4h99UJvo89HB'}],
   'available_markets': ['AD',
    'AE',
    'AG',
    'AL',
    'AM',
    'AO',
    'AR',
    'AT',
    'AU',
    'AZ',
    'BA',
    'BB',
    'BD',
    'BE',
    'BF',
    'BG',
    'BH',
    'BI',
    'BJ',
    'BN',
    'BO',
    'BR',
    'BS',
    'BT',
    'BW',
    'B

In [15]:
len(test['items'])

1

In [16]:
test['items'][0].keys()

dict_keys(['artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_urls', 'href', 'id', 'is_local', 'name', 'preview_url', 'track_number', 'type', 'uri'])

### figure out how to get track IDs from each of album avoe of the above

In [23]:
def tracks_from_album(df):
    data = {}
    data['album_id'] = []
    data['track_id'] = []
    data['track_name'] = []
    data['artist_id_1'] = []
    data['artist_id_2'] = []
    data['artist_ids'] = []
    data['artist_names'] = []
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))
    for i,x in df.iterrows():
        album_id = x['id']
        track_data = sp.album_tracks(album_id)
        items = track_data['items']

        for item in items:

            # map 1:1 data for each record
            data['album_id'].append(album_id)
            data['track_id'].append(item['id'])
            data['track_name'].append(item['name'])

            # get artist(s) data
            artist_lst = item['artists']
            artist_ids = [x['id'] for x in artist_lst]
            data['artist_id_1'].append(artist_ids[0])
            if len(artist_ids) > 1:
                data['artist_id_2'].append(artist_ids[1])
            else:
                data['artist_id_2'].append('')
            data['artist_ids'].append(str(artist_ids))
            data['artist_names'].append(str([x['name'] for x in artist_lst]))
    
    return pd.DataFrame.from_dict(data,orient='columns')

In [24]:
%%time
tracks_df = tracks_from_album(album_df)
tracks_df.head()

CPU times: user 74.7 ms, sys: 6.28 ms, total: 80.9 ms
Wall time: 1.56 s


,album_id,track_id,track_name,artist_id_1,artist_id_2,artist_ids,artist_names
0,7jMo8dhMIorHoLlzTisjJy,6KgtcmCF9Ky68XC7ezxl3s,Rumors (feat. Cardi B),56oDRnqbIiwx4mymNEv7dS,4kYSro6naA4h99UJvo89HB,"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo...","['Lizzo', 'Cardi B']"
1,2TvfE8CY37OQIPVGcWYpEA,0bI7K9Becu2dtXK1Q3cZNB,Don't Be Shy,2o5jDhtHVPhrJdv3cEQ99Z,790FomKkXshlbRYZFtlgla,"['2o5jDhtHVPhrJdv3cEQ99Z', '790FomKkXshlbRYZFt...","['Tiësto', 'KAROL G']"
2,5D8Rdb09BkmHscEGSWAlA6,6zSpb8dQRaw0M1dK8PBwQz,Cold Heart - PNAU Remix,3PhoLpVuITZKcymswpck5b,6M2wZ9GZgrQXHCFfjv46we,"['3PhoLpVuITZKcymswpck5b', '6M2wZ9GZgrQXHCFfjv...","['Elton John', 'Dua Lipa', 'PNAU']"
3,00fcDjvEq4elj756TESO0c,6OTKVgVpVaVjhRLYizPJKA,Just for me,78rUTD7y6Cy67W1RVzYs7t,,['78rUTD7y6Cy67W1RVzYs7t'],['PinkPantheress']
4,1qMFjBarjO2xD15BwXZguD,5hiZJE6Fg14Wao6EJ0KUC8,West Hills,0C0XlULifJtAgn6ZNCW2eu,,['0C0XlULifJtAgn6ZNCW2eu'],['The Killers']


In [25]:
tracks_df

,album_id,track_id,track_name,artist_id_1,artist_id_2,artist_ids,artist_names
0,7jMo8dhMIorHoLlzTisjJy,6KgtcmCF9Ky68XC7ezxl3s,Rumors (feat. Cardi B),56oDRnqbIiwx4mymNEv7dS,4kYSro6naA4h99UJvo89HB,"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo...","['Lizzo', 'Cardi B']"
1,2TvfE8CY37OQIPVGcWYpEA,0bI7K9Becu2dtXK1Q3cZNB,Don't Be Shy,2o5jDhtHVPhrJdv3cEQ99Z,790FomKkXshlbRYZFtlgla,"['2o5jDhtHVPhrJdv3cEQ99Z', '790FomKkXshlbRYZFt...","['Tiësto', 'KAROL G']"
2,5D8Rdb09BkmHscEGSWAlA6,6zSpb8dQRaw0M1dK8PBwQz,Cold Heart - PNAU Remix,3PhoLpVuITZKcymswpck5b,6M2wZ9GZgrQXHCFfjv46we,"['3PhoLpVuITZKcymswpck5b', '6M2wZ9GZgrQXHCFfjv...","['Elton John', 'Dua Lipa', 'PNAU']"
3,00fcDjvEq4elj756TESO0c,6OTKVgVpVaVjhRLYizPJKA,Just for me,78rUTD7y6Cy67W1RVzYs7t,,['78rUTD7y6Cy67W1RVzYs7t'],['PinkPantheress']
4,1qMFjBarjO2xD15BwXZguD,5hiZJE6Fg14Wao6EJ0KUC8,West Hills,0C0XlULifJtAgn6ZNCW2eu,,['0C0XlULifJtAgn6ZNCW2eu'],['The Killers']
...,...,...,...,...,...,...,...
78,2ZURBTlzCG2WCMuEwvRBaK,1MNVOUHuoU9xFeVfReb7sd,Lonely Love Affair,2WqEbbet6L2ndAbvhRVb2S,,['2WqEbbet6L2ndAbvhRVb2S'],['Watchhouse']
79,2ZURBTlzCG2WCMuEwvRBaK,6mQ3VQn9TEZaIuzvnkQm2I,Coming Down from Green Mountain,2WqEbbet6L2ndAbvhRVb2S,,['2WqEbbet6L2ndAbvhRVb2S'],['Watchhouse']
80,2ZURBTlzCG2WCMuEwvRBaK,4YG8qA4TCXUuWSvPS0A7bJ,Beautiful Flowers,2WqEbbet6L2ndAbvhRVb2S,,['2WqEbbet6L2ndAbvhRVb2S'],['Watchhouse']
81,2ZURBTlzCG2WCMuEwvRBaK,1tQLWFYNfmvMOkKvKUUsEC,Nightbird,2WqEbbet6L2ndAbvhRVb2S,,['2WqEbbet6L2ndAbvhRVb2S'],['Watchhouse']


### merge to get release date of album

### get audio features for every track

In [ ]:
def find_song(name, year):
  
    """
    This function returns a dataframe with data for a song given the name and release year.
    The function uses Spotipy to fetch audio features and metadata for the specified song.
    
    """
    
    song_data = defaultdict()
    results = sp.search(q= 'track: {} year: {}'.format(name,
                                                       year), limit=1)
    if results['tracks']['items'] == []:
        return None
    
    results = results['tracks']['items'][0]

    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]
    
    song_data['name'] = [name]
    song_data['year'] = [year]
    song_data['explicit'] = [int(results['explicit'])]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]
    
    for key, value in audio_features.items():
        song_data[key] = value
    
    return pd.DataFrame(song_data)
    

### get genres for every artist